In [ ]:
!pip install langchain-ollama

In [ ]:
!ollama --version

In [ ]:
#Ensuring ollama is running
!ollama serve

In [ ]:
!ollama pull llama3.2

In [ ]:
!ollama pull nomic-embed-text

In [2]:
rmdir /s /q data 

The system cannot find the file specified.


In [28]:
!pip freeze > requirements.txt

In [3]:
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [4]:
FILEPATH    = "Resume.pdf"
LOCAL_MODEL = "llama3.2"
EMBEDDING   = "nomic-embed-text"

In [5]:
loader = PyPDFLoader(FILEPATH)
data = loader.load()
#The loaded text is split into manageable chunks using RecursiveCharacterTextSplitter to facilitate efficient processing.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [6]:
persist_directory = 'data'
#Chroma vector store is generated from the split documents, utilizing Ollama embeddings for semantic representation.

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OllamaEmbeddings(model=EMBEDDING),
    persist_directory=persist_directory
)

In [7]:
#Defining Prompt Template and Memory Handling

llm = OllamaLLM(base_url="http://localhost:11434",
                                  model=LOCAL_MODEL,
                                  verbose=True,
                                  callback_manager=CallbackManager(
                                      [StreamingStdOutCallbackHandler()])
                                  )

retriever = vectorstore.as_retriever()

In [8]:
template = """ You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and to-the-point.
    
    Context: {context}
    History: {history}

    User: {question}
    Chatbot:
    """
prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=template,
    )

memory = ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question"
    )

C:\Users\vibhu\AppData\Local\Temp\ipykernel_25052\209392036.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [9]:
#The RetrievalQA chain is instantiated

qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=retriever,
            verbose=True,
            chain_type_kwargs={
                "verbose": True,
                "prompt": prompt,
                "memory": memory,
            }
        )


In [10]:
query = "What is your name?"

In [11]:
qa_chain.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and to-the-point.
    
    Context: Name: Vibhuti Dabas  
Eductaion: based in Manchester, UK. She holds an MSc in Advanced Computer Science with a 
specialization in Artificial Intelligence from the University of Manchester, expected to be completed 
in September 2025. Prior to that, she completed her Bachelor of Technology in Information 
Technology from Indira Gandhi Delhi Technical University for Women, graduating with an 8.6 CGPA in 
May 2024.

match user-uploaded images and videos with product names. 
During her time as a Machine Learning Intern at Merilrish in Tamil Nadu, India, from April to July 
2023, she worked on social media profile analysis using API data, applying sentiment analysis 
techniques to determine whether the extra

{'query': 'What is your name?', 'result': 'My name is Vibhuti Dabas.'}

In [12]:
qa_chain.invoke({"query": "What are you doing these days?"})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and to-the-point.
    
    Context: live motion analysis models to track movement postures during rehabilitation therapy sessions for 
individuals recovering from injuries. 
Projects: she has implemented a QandA system by text extraction from a given PDF, designing a 
data extraction pipeline using Ollama to structure hardware specifications from GivEnergy data 
sheets with optimized inference on single-node clusters. She also developed a voice cloning system

and deploying machine learning models for self-checkout kiosks, incorporating YOLO-based object 
detection and MMDetection to ensure real-time product recognition. She actively interacted with 
clients, handling data generation and preprocessing for model training and testing, while

{'query': 'What are you doing these days?',
 'result': "I'm doing well, thank you for asking. I've been working on various projects and tasks to improve my capabilities in the field of machine learning and data analysis. Recently, I've been assisting clients with live motion analysis models for rehabilitation therapy sessions, as well as deploying voice cloning systems for self-checkout kiosks. I'm also actively engaged in optimizing inference on edge devices for real-time product recognition and other applications.\n\nIn my previous roles as a Data Science Intern at CRUV Dimensions Pvt. Ltd. and Merilrish, I worked on projects such as capturing shopping cart screenshots from food apps, social media profile analysis using API data, and developing one-shot learning models to match user-uploaded images with product names. My experience has been valuable in helping me stay up-to-date with the latest advancements in machine learning and data analysis.\n\nI'm always looking for new opportun

In [13]:
qa_chain.invoke({"query": "What is your experience in AI jobs?"})



> Entering new RetrievalQA chain...




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and to-the-point.
    
    Context: Intelligence Application (AIA) journal with a study on developing a university auto-reply FAQ chatbot 
using NLP and neural networks. 
Professional Experience: She has worked as a Jr. AI Engineer at Xplorazzi Adventures Pvt. Ltd. in 
Bengaluru, India, from March to October 2024. In this role, she played a crucial role in developing 
and deploying machine learning models for self-checkout kiosks, incorporating YOLO-based object

and deploying machine learning models for self-checkout kiosks, incorporating YOLO-based object 
detection and MMDetection to ensure real-time product recognition. She actively interacted with 
clients, handling data generation and preprocessing for model training and testing, while also 
optimizing inference on edge

{'query': 'What is your experience in AI jobs?',
 'result': 'As a Jr. AI Engineer with 2 years of experience, I have worked on various AI-related projects and tasks across different domains. My professional experience includes:\n\n- Developing machine learning models for self-checkout kiosks at Xplorazzi Adventures Pvt. Ltd., where I incorporated YOLO-based object detection and MMDetection to ensure real-time product recognition.\n- Working as a Data Science Intern at CRUV Dimensions Pvt. Ltd. and Merilrish, where I worked on projects such as capturing shopping cart screenshots from food apps and developing one-shot learning models for user-uploaded images and videos matching with product names.\n- Completing an internship at VLink Inc., where I gained experience in deep learning techniques using a Bi-LSTM model.\n\nMy technical expertise spans across Machine Learning & AI, including regression, classification, deep learning frameworks such as TensorFlow, PyTorch, and Keras, and large 

In [17]:
qa_chain.invoke({"query": "What are your research contributions?"})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and to-the-point.
    
    Context: May 2024. 
Research publication paper: Her research contributions include the construction of hyperspectral 
images from RGB images using CNN, published in the Multimedia Tools and Applications journal. This 
study explored an affordable approach to converting RGB images into Hyperspectral Images (HSI), 
making medical and environmental monitoring more accessible. She also contributed to the Artificial

match user-uploaded images and videos with product names. 
During her time as a Machine Learning Intern at Merilrish in Tamil Nadu, India, from April to July 
2023, she worked on social media profile analysis using API data, applying sentiment analysis 
techniques to determine whether the extracted text 

{'query': 'What are your research contributions?',
 'result': 'Based on the provided context, Vibhuti Dabas has made a significant research contribution in constructing hyperspectral images from RGB images using Convolutional Neural Networks (CNN). This study was published in the Multimedia Tools and Applications journal and aims to provide an affordable approach to converting RGB images into Hyperspectral Images (HSI), making medical and environmental monitoring more accessible.'}